<a href="https://colab.research.google.com/github/cypision/Alchemy-in-MLDL/blob/master/word_embedding_add_oob_Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__웰니스_대화_스크립트_데이터셋 Embedding 하기__

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install konlpy
!pip install jpype1==0.7.0

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 59.6MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
     |████████████████████████████████| 2.7MB 12.2MB/s 
  Found existing installation: JPype1 1.2.0
    Uninstalling JPype1-1.2.0:
      Successfully uninstalled JPype1-1.2.0


In [7]:
import numpy as np
import tensorflow as tf
import json
import random
import pandas as pd
random.seed(1)

In [8]:
EXCEL_FILE_NALE = "/content/웰니스_대화_스크립트_데이터셋.xlsx"
data = pd.read_excel(EXCEL_FILE_NALE)

In [9]:
DATA = []
RESPONSE = {}
 
for i in range(len(data["구분"])):
  label = data["구분"][i]
  label_split = label.split("/")
  label_1 = "/".join(label_split[:2])
  sent = data["유저"][i]
  if label_1 != "모호함":
    DATA.append(["Sent_{}".format(i), sent, label_1, label])
    if label_1 in RESPONSE:  
      if not pd.isnull(data["챗봇"][i]): 
        RESPONSE[label_1].append(data["챗봇"][i])
    else: 
      if not pd.isnull(data["챗봇"][i]):  
        RESPONSE[label_1] = [data["챗봇"][i]]
 
"""random shuffle & make them into train/test set"""
labels = [dat[2] for dat in DATA]

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(DATA, random_state = 2020, stratify = labels, test_size = 400)

In [11]:
print("Data Example")
for i in range(5):
  print(train[i])

Data Example
['Sent_4393', '뭔가 하루종일 이렇게 들뜬 기분이다 보니까 잠도 잘 안 와.', '증상/불면', '증상/불면']
['Sent_603', '아무한테나 화내고 그러지는 않아.', '감정/분노', '감정/분노']
['Sent_4224', '잠자리에 누워도 맨날 뒤척이고... 잠을 제대로 잘 수 있을 리가 없지.', '증상/불면', '증상/불면']
['Sent_3849', '5일 전에는 새벽에 일어나서 화장실을 가다가 순간적으로 정신을 잃었어.', '증상/기절', '증상/기절']
['Sent_666', '그냥 감정이입이 심하게 되고 불안감도 잘 느끼는 것 같아요.', '감정/불안감', '감정/불안감']


**기존 embedding layer와의 관계파악**  


In [12]:
from konlpy.tag import Komoran, Hannanum, Kkma, Okt ## using only Komoran object

## 형태소분석 함수 만들기
komoran = Komoran()
def tokenize(word):
  return komoran.morphs(word)

**기존 Ebedding layer가져오기**

In [13]:
import numpy as np
with open("/content/gdrive/My Drive/CNS_NLP/vecs.tsv") as f:
  vecs = [v.strip() for v in f.readlines()]
final_embeddings = [v.split("\t") for v in vecs]
final_embeddings = np.array(final_embeddings, dtype="float32")
with open("/content/gdrive/My Drive/CNS_NLP/meta.tsv") as f:
  vocab_list = [v.strip() for v in f.readlines()]


print(final_embeddings.shape)
print(len(vocab_list))

(70002, 128)
70002


**신규 sentece들을 tokenize 했을때, 기존 meta token들과의 관계 파악하기**  

In [14]:
import collections
tot_tokens = 0
oov_counter = collections.Counter()
Tokenized_train = []

In [15]:
for data_lst in train:
  sentence = data_lst[1]
  tokenized_sent = tokenize(sentence)
  tot_tokens += len(tokenized_sent) ## 전체 tocken 수 count 중복허용

  for word in tokenized_sent:
    if word not in vocab_list:
      oov_counter[word] += 1
  Tokenized_train.append([data_lst[0], tokenized_sent, data_lst[2]])

In [16]:
print("# OOV Tokens:", len(oov_counter))
# print("{}/{} ({:.2f}%) are [UNK] in train tokens".format(sum(oov_counter.values()) , tot_tokens , 100*sum(oov_counter.values())/tot_tokens))

# OOV Tokens: 471


새로 등장한 토큰은 471 개이다.

In [17]:
### 새로 나온 데이터 살펴보기
most_common = oov_counter.most_common(len(oov_counter))
print(most_common[:10])
print(most_common[-10:])
 
### Train 데이터에서 새로 발견한 토큰을 기존 단어 사전에 추가
new_vocab_list = vocab_list.copy()
new_vocab_list.extend([v[0] for v in most_common])
print("\n# New Vocabs = {}".format(len(new_vocab_list)))
 
##TEMP 신규로 추가한 단어 사전
print(new_vocab_list[-30:])

[('걔', 37), ('더라구', 20), ('어떡하지', 17), ('여치', 13), ('대요', 13), ('두근거리', 13), ('뒤척이', 12), ('갑갑', 12), ('아파서', 10), ('얼른', 8)]
[('버럭', 1), ('아차', 1), ('싱숭생숭하네요…', 1), ('대견', 1), ('돌덩이', 1), ('쭈꾸미를', 1), ('혹시나', 1), ('헷갈려요.', 1), ('근육통', 1), ('볼링공', 1)]

# New Vocabs = 70473
['과태료', '피시방', '얼결', '울렁거려요.', '부들부들', '흥청망청', '둥둥', '째려보', '까마득', '흔들다리', '갓길', '틈틈히', '뒷머리', '뒷목', '짓누르', '예뻐서', '스르륵', '꿈쩍', '홀딱', '문제집', '버럭', '아차', '싱숭생숭하네요…', '대견', '돌덩이', '쭈꾸미를', '혹시나', '헷갈려요.', '근육통', '볼링공']


In [18]:
print(len(new_vocab_list)) ## 기존 70002 -> 70473 으로 늘어났다.

70473


In [19]:
## show sample data and compare before vs after
print(len(Tokenized_train),len(train))
print(train[0],"\n=============================================================================================================\n",Tokenized_train[0])

4651 4651
['Sent_4393', '뭔가 하루종일 이렇게 들뜬 기분이다 보니까 잠도 잘 안 와.', '증상/불면', '증상/불면'] 
 ['Sent_4393', ['뭐', 'ㄴ가', '하루', '종일', '이렇', '게', '들뜨', 'ㄴ', '기분', '이', '다', '보', '니까', '잠도', '잘', '안', '오', '아', '.'], '증상/불면']


**train,test 에 있는 데이터들을 embedding 하기**  


In [20]:
from utils import TextEncoder
text_encoder = TextEncoder(new_vocab_list) ## new_vocab_list 기준으로 dictionary를 만듬.

__TextEncoder__  
- TextEncoder 이전에 만든 객체임  
- new_vocab_list 기준으로 만들었으며, token_to_id , id_to_tocken 기능을 수행한다  

In [21]:
# train_ids, train_labels, label_map = create_cls_feature(Tokenized_train, text_encoder, max_seq_len=MAX_LEN, )
label_map=None
# examples=Tokenized_train
text_encoder=text_encoder
max_seq_len=None

In [22]:
input_ids = [] # 정수 인덱스로 변환한 문장들의 리스트
labels = [] # 정답 라벨 리스트
MAX_LEN = 50 # sequence 길이

In [23]:
# if label_map is None: #label_map이 없으면 -> 이번에 데이터를 처리하면서 새로 생성함
#     CREATE_LABEL_MAP = True
#     label_map = {}
#     label_index = 0
# else: 
#     CREATE_LABEL_MAP = False
#     print("** Start creating features using label map")
#     print(label_map)

**실제 처리 과정**

In [24]:
# CREATE_LABEL_MAP = True ?
label_map = {}
label_index = 0

In [29]:
for example in Tokenized_train:
    idx, tokenized_sent, label = example

    ## 1. text_encoder 사용해 정수로 변환 
    input_id = text_encoder.convert_tokens_to_ids(tokenized_sent)
    if len(input_id) == 0: ## list로 return 받은 정수 인덱스들의 길이가 0 이며. 즉 없으면.
        print("Sentence with length = 0... continue", example)
        continue

    ## 2. label 매핑 & index로 변환
    if label in label_map: ## label이 label_map에 key로 등록되어 있으면 = label이 신규가 아니면
        label_id = label_map[label]
    else:
      # label map에 추가
      label_map[label] = label_index
      label_index += 1
      label_id = label_map[label]        

    ## 전체 리스트에 append
    input_ids.append(input_id)
    labels.append(label_id)

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

""" max_seq_len을 넘는 문장은 절단, 모자르는 것은 PADDING """
input_ids = pad_sequences(input_ids,maxlen=MAX_LEN,padding="post",truncating="pre")

## np.array로 변환해 리턴
input_ids = np.array(input_ids)
labels = np.array(labels)

assert len(input_ids) == len(labels)
print("** {} examples processed".format(len(input_ids)))

** 4651 examples processed


In [31]:
print(len(input_ids[0]))
input_ids[0]

50


array([ 4031,  1476,  1453, 13158,   497,    40, 14785,     9,  5861,
           3,     8,    85,  4852, 29044,   359,   163,   101,    18,
           4,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)

In [32]:
for i in range(len(input_ids)):
  if len(input_ids[i]) > 50: ## max_sequence = 50 을 넘는 값이 있는지 확인. 당연히 없다.
    print(input_ids[i])

In [33]:
print(len(Tokenized_train))

4651


In [40]:
train_ids = input_ids.copy(); train_labels = labels.copy()

label_map_reverse = {}
for key,val in label_map.items():
  label_map_reverse[val] = key

print(train_ids[0])
print(train_labels[0])
print(len(label_map)) ## dict type이므로 key값을 넣어야 한다.
print(label_map_reverse[train_labels[0]]) ## dict type이므로 key값을 넣어야 한다.

[ 4031  1476  1453 13158   497    40 14785     9  5861     3     8    85
  4852 29044   359   163   101    18     4     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
0
176
증상/불면


input_ids 은 Tokenized_train 값들이 문장길이 Maxlen = 50 사이즈로, index화 된 값이다.  
input 값들이 Embedding 계층을 통과하면, Embedding 된 벡터(단어)들로 학습되기 된다.

In [39]:
label_map.keys()

dict_keys(['증상/불면', '감정/분노', '증상/기절', '감정/불안감', '증상/두근거림', '배경/건강문제', '배경/취업', '감정/우울감', '배경/대인관계', '배경/어린시절', '감정/자살충동', '감정/외로움', '증상/자살시도', '감정/걱정', '배경/학업', '증상/무기력', '증상/식욕저하', '일반대화', '배경/결혼', '상태/증상지속', '배경/학교', '감정/짜증', '배경/부모', '증상/두통', '증상/피해망상', '감정/자존감저하', '배경/성격', '증상/기억상실', '배경/사업', '감정/감정조절이상', '배경/전연인', '감정/불편감', '배경/경제적문제', '치료이력/병원내원', '배경/친구', '배경/가족', '배경/여자친구', '치료이력/검사', '증상/어지러움', '감정/부정적사고', '감정/눈물', '배경/시댁', '증상/반복행동', '감정/좌절', '배경/자녀', '감정/무력감', '배경/사고', '배경/직장', '증상/기억력저하', '증상/은둔', '증상/환청', '증상/이명', '부가설명', '감정/심란', '감정/답답', '감정/힘듦', '감정/공허감', '배경/남편', '증상/반복사고', '감정/무서움', '감정/두려움', '증상/피로', '감정/속상함', '감정/통제력상실', '배경/생활', '배경/종교', '감정/자괴감', '증상/환각', '배경/음주', '배경/대학', '자가치료/심리조절', '감정/당황', '감정/즐거움', '배경/남자친구', '증상/생리불순', '감정/불쾌감', '증상/공황발작', '감정/억울함', '감정/배신감', '증상/호흡곤란', '감정/신경쓰임', '배경/문제', '배경/귀국', '배경/애완동물', '배경/타인', '감정/절망감', '증상/이인감', '증상/대화기피', '감정/서운함', '내원이유/치료', '감정/괴로움', '치료이력/응급실', '감정/자신감저하', '감정/화', '감정/충격', '증상/집중력저하', '증상/자해', '배경/이사', '배경/연애',

In [42]:
print(train_ids.shape)
print(train_labels.shape)

(4651, 50)
(4651,)


__Test 데이터에 대해, 토크나이즈 하고, Embedding 하기__

In [27]:
## 토크나이즈
Tokenized_test = []

for data_lst in test:
  sentence = data_lst[1]
  tokenized_sent = tokenize(sentence)
  
  for word in tokenized_sent:
    if word not in vocab_list:
      oov_counter[word] += 1
  Tokenized_test.append([data_lst[0], tokenized_sent, data_lst[2]])

In [28]:
print(len(test),len(Tokenized_test))
Tokenized_test[0]

400 400


['Sent_1296', ['저', '는', '이제', '망하', '았', '어요', '…'], '감정/좌절']

In [43]:
from utils import TextEncoder,create_cls_feature

test_ids, test_labels, _ = create_cls_feature(Tokenized_test, text_encoder, max_seq_len=50, label_map = label_map)

** Start creating features using label map
{'증상/불면': 0, '감정/분노': 1, '증상/기절': 2, '감정/불안감': 3, '증상/두근거림': 4, '배경/건강문제': 5, '배경/취업': 6, '감정/우울감': 7, '배경/대인관계': 8, '배경/어린시절': 9, '감정/자살충동': 10, '감정/외로움': 11, '증상/자살시도': 12, '감정/걱정': 13, '배경/학업': 14, '증상/무기력': 15, '증상/식욕저하': 16, '일반대화': 17, '배경/결혼': 18, '상태/증상지속': 19, '배경/학교': 20, '감정/짜증': 21, '배경/부모': 22, '증상/두통': 23, '증상/피해망상': 24, '감정/자존감저하': 25, '배경/성격': 26, '증상/기억상실': 27, '배경/사업': 28, '감정/감정조절이상': 29, '배경/전연인': 30, '감정/불편감': 31, '배경/경제적문제': 32, '치료이력/병원내원': 33, '배경/친구': 34, '배경/가족': 35, '배경/여자친구': 36, '치료이력/검사': 37, '증상/어지러움': 38, '감정/부정적사고': 39, '감정/눈물': 40, '배경/시댁': 41, '증상/반복행동': 42, '감정/좌절': 43, '배경/자녀': 44, '감정/무력감': 45, '배경/사고': 46, '배경/직장': 47, '증상/기억력저하': 48, '증상/은둔': 49, '증상/환청': 50, '증상/이명': 51, '부가설명': 52, '감정/심란': 53, '감정/답답': 54, '감정/힘듦': 55, '감정/공허감': 56, '배경/남편': 57, '증상/반복사고': 58, '감정/무서움': 59, '감정/두려움': 60, '증상/피로': 61, '감정/속상함': 62, '감정/통제력상실': 63, '배경/생활': 64, '배경/종교': 65, '감정/자괴감': 66, '증상/환각': 67, '배경/음주': 68, '배경/대학

In [45]:
print(len(test_ids),len(test_labels))
print(test_ids[0])
print(test_labels[0])
print(len(label_map)) ## dict type이므로 key값을 넣어야 한다.
print(label_map_reverse[test_labels[0]]) ## dict type이므로 key값을 넣어야 한다.

400 400
[  594     6  1549 12031    12  1365  2223     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
43
176
감정/좌절


In [46]:
print(test_ids.shape,test_labels.shape)

(400, 50) (400,)


토근화까지 진행된 데이터들을 저장한다.

In [48]:
# with open("/content/gdrive/My Drive/CNS_NLP/Wellness_data_train_tokenized.json","w") as f:
#   f.write(json.dumps(train_ids))
# with open("/content/gdrive/My Drive/CNS_NLP/Wellness_data_test_tokenized.json","w") as f:
#   f.write(json.dumps(test_ids))
# with open("/content/gdrive/My Drive/CNS_NLP/Wellness_data_train_tokenized_label.json","w") as f:
#   f.write(json.dumps(train_labels))
# with open("/content/gdrive/My Drive/CNS_NLP/Wellness_data_test_tokenized_label.json","w") as f:
#   f.write(json.dumps(test_labels))

## label_map
with open("/content/gdrive/My Drive/CNS_NLP/Wellness_data_label_map.json","w") as f:
  f.write(json.dumps(label_map))

In [49]:
## numpy 배열 저장
np.save('/content/gdrive/My Drive/CNS_NLP/Wellness_data_train_tokenized',train_ids)
np.save('/content/gdrive/My Drive/CNS_NLP/Wellness_data_test_tokenized',test_ids)
np.save('/content/gdrive/My Drive/CNS_NLP/Wellness_data_train_tokenized_label',train_labels)
np.save('/content/gdrive/My Drive/CNS_NLP/Wellness_data_test_tokenized_label',test_labels)